In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [10]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 10  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load nci


In [5]:
drug_sum

array([38., 35., 34., ..., 23., 24., 25.], dtype=float32)

In [6]:
cell_sum

array([456., 593., 453., 532., 323., 567., 618., 499., 383., 172., 239.,
       654., 345., 641., 452., 586., 412., 528., 432., 434., 499., 704.,
       497., 408., 715., 258., 466.,  82., 620., 252., 355., 445., 241.,
       613., 576., 318., 221., 204., 360., 356., 444., 592., 375., 586.,
       615., 297., 223., 539., 389., 486., 280., 666., 701., 503., 451.,
       328., 525., 252., 622., 554.], dtype=float32)

In [11]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [12]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = nihgcn_new(
                cell_exprs=exprs,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  2%|▏         | 1/60 [00:00<00:19,  3.09it/s]

epoch:   0 loss:0.699949 auc:0.4734
Fit finished.


  3%|▎         | 2/60 [00:00<00:18,  3.11it/s]

epoch:   0 loss:0.702829 auc:0.5612
Fit finished.


  5%|▌         | 3/60 [00:01<00:21,  2.61it/s]

...

epoch:   0 loss:0.702465 auc:0.4839
Fit finished.


 98%|█████████▊| 59/60 [00:20<00:00,  4.45it/s]

epoch:   0 loss:0.703517 auc:0.4030
Fit finished.


100%|██████████| 60/60 [00:20<00:00,  2.93it/s]

epoch:   0 loss:0.701695 auc:0.5129
Fit finished.


In [13]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")

In [ ]:
true_data_s